In [23]:
import pandas as pd
import requests

In [51]:
# amsterdam tree data
df_trees = pd.read_csv('data/bomen-stamgegevens-2024-03-12T09 06 04.036291.csv')
# amsterdam neighbourhood meta data, i.e. matching neighbourhood name with ID
df_buurt_meta = pd.read_csv('data/gebieden-buurten-2024-03-12T12 09 07.653588.csv')

# standardise names of neighbourhood id column so we can merge on that column
neighbourhood_id_column = 'neighbourhood_id'

df_buurt_meta = df_buurt_meta.rename(
    columns={'Identificatie': neighbourhood_id_column}
)
df_trees = df_trees.rename(
    columns={'Gbdbuurtid': neighbourhood_id_column}
)


In [55]:
df = pd.merge(df_trees, df_buurt_meta, on='neighbourhood_id')